In [1]:
import glob
import SimpleITK as sitk
import numpy as np
import collections
from PIL import Image, ImageDraw
#from segmentation import segment
import os
import csv
import ast
import collections
from collections import namedtuple
import random
from pylidc_func import create_ann_malcsv, create_ann_maldf
#from cmask_dict import cmask_dict


In [2]:
isLungSegmentProcess = False

In [3]:
XyzTuple = collections.namedtuple('XyzTuple', ['x', 'y', 'z'])
IrcTuple = collections.namedtuple('IrcTuple', ['index', 'row', 'col'])
def xyz2irc(coord_xyz, origin_xyz, vxSize_xyz, direction_a):
    origin_a = np.array(origin_xyz)
    vxSize_a = np.array(vxSize_xyz)
    coord_a = np.array(coord_xyz)
    cri_a = ((coord_a - origin_a) @ np.linalg.inv(direction_a)) / vxSize_a
    cri_a = np.round(cri_a)
    return IrcTuple(int(cri_a[2]), int(cri_a[1]), int(cri_a[0]))


In [4]:
# df_mal = create_ann_maldf()
# create_ann_malcsv(df_mal)

In [5]:
#print(cmask_dict)

In [6]:
# import json 

# with open('cmask_dict.py', 'w') as file:
#     file.write(str("dict = ").rstrip('\n'))
# 	file.write(str(cmask_dict))


In [7]:
CandidateInfoTuple = namedtuple('CandidateInfoTuple', 'isNodule_bool, hasAnnotation_bool, isMal_bool, series_uid, center_xyz, cbbox, mask_box')
def getCandidateInfoList(requireOnDisk_bool=True):
    # We construct a set with all series_uids that are present on disk.
    # This will let us use the data, even if we haven't downloaded all of
    # the subsets yet.
    mhd_list = glob.glob('E:/LUNA/Luna16_AugData/subset*/*.mhd')
    presentOnDisk_set = {os.path.split(p)[-1][:-4] for p in mhd_list}

    candidateInfo_list = []
    with open('./annotations_object_detection_1127.csv', "r") as f:
        for row in list(csv.reader(f))[1:]:
            series_uid = row[0]
            annotationCenter_xyz = tuple([float(x) for x in row[1:4]])
            # annotationDiameter_mm = float(row[4])
            isMal_bool = {'0.0': False, '1.0': True}[row[5]] #it record the malignancy or not
            cbbox = ast.literal_eval(row[7])
            mask_box = row[8]
            mask_list = ast.literal_eval(mask_box)
            
            if series_uid not in presentOnDisk_set and requireOnDisk_bool:
                continue

            candidateInfo_list.append(
                CandidateInfoTuple(
                    True,
                    True,
                    isMal_bool,
                    series_uid,
                    annotationCenter_xyz,
                    cbbox,
                    mask_list
                )
            )
    candidateInfo_list.sort(reverse=True)
    return candidateInfo_list


In [8]:
candidateInfo_list = getCandidateInfoList()
len(candidateInfo_list)


1182

In [9]:
candidateInfo_list[0]


CandidateInfoTuple(isNodule_bool=True, hasAnnotation_bool=True, isMal_bool=True, series_uid='1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084993415992563148335', center_xyz=(-21.95847826, 33.48609566, -155.2920257), cbbox=[116, 140], mask_box=[[0.4452054795, 0.5870841487, 0.013671875, 0.0234375], [0.4442270059, 0.5851272016, 0.03125, 0.03515625], [0.4471624266, 0.5684931507, 0.052734375, 0.072265625], [0.4481409002, 0.5743639922, 0.05859375, 0.080078125], [0.4500978474, 0.5772994129, 0.06640625, 0.08203125], [0.4442270059, 0.5802348337, 0.05859375, 0.076171875], [0.4442270059, 0.584148728, 0.0625, 0.068359375], [0.4432485323, 0.5851272016, 0.060546875, 0.06640625], [0.4442270059, 0.584148728, 0.05859375, 0.068359375], [0.4442270059, 0.5831702544, 0.05859375, 0.06640625], [0.4432485323, 0.5890410959, 0.060546875, 0.07421875], [0.4432485323, 0.5909980431, 0.064453125, 0.078125], [0.446183953, 0.5900195695, 0.0703125, 0.080078125], [0.4471624266, 0.5909980431, 0.076171875, 0.0859375], [0.44

In [10]:
mal_count_list = {'Benign': 0, 'Malignancy': 0, 'nan': 0}
for candidate in candidateInfo_list:
    if candidate[2]:
        for item in candidate[6]:
            if item == 'nan':
                mal_count_list['nan'] += 1
            else:
                mal_count_list['Malignancy'] += 1
    else:
         for item in candidate[6]:
            if item == 'nan':
                mal_count_list['nan'] += 1
            else:
                mal_count_list['Benign'] += 1

print(mal_count_list)

generate = {'Train': {'Benign': mal_count_list['Benign'] - 2 * int(mal_count_list['Benign'] * 0.1), 'Malignancy': mal_count_list['Malignancy'] - 2 * int(mal_count_list['Malignancy'] * 0.1)}, 'Valid': {'Benign':  int(mal_count_list['Benign'] * 0.1), 'Malignancy': int(mal_count_list['Malignancy'] * 0.1)}, 'Test': {'Benign':  int(mal_count_list['Benign'] * 0.1), 'Malignancy':  int(mal_count_list['Malignancy'] * 0.1)}}
print(generate)

{'Benign': 4117, 'Malignancy': 3788, 'nan': 966}
{'Train': {'Benign': 3295, 'Malignancy': 3032}, 'Valid': {'Benign': 411, 'Malignancy': 378}, 'Test': {'Benign': 411, 'Malignancy': 378}}


In [11]:
# def buildAnnotationMaskNew(noduleInfo_list, mask):
#     for candidateInfo_tup in noduleInfo_list:
#         bbox_info = 
        
        

In [12]:
def buildAnnotationMask(noduleInfo_list, hu_a, origin_xyz, vxSize_xyz, direction_a, threshold_hu = -700):
        boundingBox_a = np.zeros_like(hu_a, dtype=bool) # all False tensor
        bbox_result = []
        for candidateInfo_tup in noduleInfo_list: #traverse所有的nodules
            center_irc = xyz2irc(
                candidateInfo_tup.center_xyz,
                origin_xyz,
                vxSize_xyz,
                direction_a,
            )
            ci = int(center_irc.index)  # the center of voxel
            cr = int(center_irc.row)
            cc = int(center_irc.col)

            index_radius = 2
            try:
                # 從index找，看哪裡會遇到空氣，當兩邊都遇到空氣後就把邊界設在大的那段
                while hu_a[ci + index_radius, cr, cc] > threshold_hu and \
                        hu_a[ci - index_radius, cr, cc] > threshold_hu:
                    index_radius += 1
            except IndexError:
                index_radius -= 1

            row_radius = 2
            try:
                # 從row找，看哪裡會遇到空氣，當兩邊都遇到空氣後就把邊界設在大的那段
                while hu_a[ci, cr + row_radius, cc] > threshold_hu and \
                        hu_a[ci, cr - row_radius, cc] > threshold_hu:
                    row_radius += 1
            except IndexError:
                row_radius -= 1

            col_radius = 2
            try:
                # 從column找，看哪裡會遇到空氣，當兩邊都遇到空氣後就把邊界設在大的那段
                while hu_a[ci, cr, cc + col_radius] > threshold_hu and \
                        hu_a[ci, cr, cc - col_radius] > threshold_hu:
                    col_radius += 1
            except IndexError:
                col_radius -= 1

            # assert index_radius > 0, repr([candidateInfo_tup.center_xyz, center_irc, self.hu_a[ci, cr, cc]])
            # assert row_radius > 0
            # assert col_radius > 0

            boundingBox_a[
                 ci - index_radius: ci + index_radius + 1,
                 cr - row_radius: cr + row_radius + 1,
                 cc - col_radius: cc + col_radius + 1] = True #將box裡的所有格子設成TRUE
            
            bbox_result.append([round((abs(cr) + 1) / 512, 10), round((abs(cc) + 1) / 512, 10), round(2 * row_radius / 512, 10), round(2 * col_radius / 512, 10)])

        #mask_a = boundingBox_a & (hu_a > threshold_hu)  #最後會對box和threshold低於-700的值做and
        mask_a = boundingBox_a
        return mask_a, bbox_result


In [13]:
def getCandidateInfoDict(requireOnDisk_bool=True):  #把candidateInfoList包成Dict
    candidateInfo_list = getCandidateInfoList(requireOnDisk_bool)
    candidateInfo_dict = {}

    for candidateInfo_tup in candidateInfo_list:
        candidateInfo_dict.setdefault(candidateInfo_tup.series_uid,
                                      []).append(candidateInfo_tup)

    return candidateInfo_dict


In [14]:
candidate_dict =  getCandidateInfoDict()
len(candidate_dict)


600

In [15]:
print(candidate_dict)

{'1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084993415992563148335': [CandidateInfoTuple(isNodule_bool=True, hasAnnotation_bool=True, isMal_bool=True, series_uid='1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084993415992563148335', center_xyz=(-21.95847826, 33.48609566, -155.2920257), cbbox=[116, 140], mask_box=[[0.4452054795, 0.5870841487, 0.013671875, 0.0234375], [0.4442270059, 0.5851272016, 0.03125, 0.03515625], [0.4471624266, 0.5684931507, 0.052734375, 0.072265625], [0.4481409002, 0.5743639922, 0.05859375, 0.080078125], [0.4500978474, 0.5772994129, 0.06640625, 0.08203125], [0.4442270059, 0.5802348337, 0.05859375, 0.076171875], [0.4442270059, 0.584148728, 0.0625, 0.068359375], [0.4432485323, 0.5851272016, 0.060546875, 0.06640625], [0.4442270059, 0.584148728, 0.05859375, 0.068359375], [0.4442270059, 0.5831702544, 0.05859375, 0.06640625], [0.4432485323, 0.5890410959, 0.060546875, 0.07421875], [0.4432485323, 0.5909980431, 0.064453125, 0.078125], [0.446183953, 0.5900195695, 0.0703125, 0.0

In [16]:
def np2Png(np_arr, target_path):
    min_value = np.min(np_arr)
    max_value = np.max(np_arr) if np.max(np_arr) > 0 else 1
    scaled_np_arr = (np_arr - min_value) / (max_value - min_value) * 255
    scaled_np_arr = scaled_np_arr.astype(np.uint8)
    slices = Image.fromarray(scaled_np_arr, mode='L')
    slices = slices.resize((640, 640))
    slices.save(target_path, cmap='gray')


In [17]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image 
from matplotlib.patches import Rectangle

def np2MarkPng(np_arr, target_path, row, col):
    min_value = np.min(np_arr)
    max_value = np.max(np_arr)
    scaled_np_arr = (np_arr - min_value) / (max_value - min_value) * 255
    scaled_np_arr = scaled_np_arr.astype(np.uint8)
    slices = Image.fromarray(scaled_np_arr, mode='L')
    draw = ImageDraw.Draw(slices)
    x1, y1, x2, y2 = col-10, row-10, col+10, row+10
    draw.rectangle([x1, y1, x2, y2], outline="white", width=1)
    slices.save(target_path)
    im = Image.open(target_path)

    # Create figure and axes
    fig, ax = plt.subplots()

    # Display the image
    ax.imshow(im)

def bboxOnImg(img_path, bbox):
    [x, y, height, width] = bbox
    
    x *= 640
    y *= 640
    height *= 640
    width *= 640
    
    im2 = Image.open(img_path)

    # Create figure and axes
    fig2, ax2 = plt.subplots()

    # Display the image
    ax2.imshow(im2)
    size = im2.size
    print(size)

    # Create a Rectangle patch
    rect = Rectangle((x-height/2, y - width/2), height, width, linewidth=1, edgecolor='r', facecolor='none')

    # Add the patch to the Axes
    ax2.add_patch(rect)

    plt.show()
    

   


In [18]:
import torch
import torch.nn as nn
import numpy as np
from torchvision import datasets, transforms, models
from torch.utils.data import ConcatDataset, DataLoader, Subset, Dataset
import scipy
from scipy.ndimage import binary_erosion, binary_dilation, binary_fill_holes
from collections import deque
import copy
import matplotlib.pyplot as plt
from torchvision.models import ResNet34_Weights
from tqdm import tqdm
from PIL import Image, ImageDraw
from scipy.spatial import ConvexHull
from bresenham import bresenham


class ConvLSTMCell(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, padding, activation, frame_size):
        super(ConvLSTMCell, self).__init__()

        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        # self.device = torch.device('cpu')

        if activation == "tanh":
            self.activation = torch.tanh
        elif activation == "relu":
            self.activation = torch.relu

        self.conv = nn.Conv2d(
            in_channels=in_channels + out_channels,
            out_channels=4 * out_channels,
            kernel_size=kernel_size,
            padding=padding)

        self.W_ci = nn.Parameter(torch.Tensor(out_channels, *frame_size))
        self.W_co = nn.Parameter(torch.Tensor(out_channels, *frame_size))
        self.W_cf = nn.Parameter(torch.Tensor(out_channels, *frame_size))

        # Initialize weights using Xavier initialization
        nn.init.xavier_uniform_(self.W_ci)
        nn.init.xavier_uniform_(self.W_co)
        nn.init.xavier_uniform_(self.W_cf)

    def forward(self, X, H_prev, C_prev):
        # print(X.shape, H_prev.shape)
        conv_output = self.conv(torch.cat([X, H_prev], dim=1))
        i_conv, f_conv, C_conv, o_conv = torch.chunk(conv_output, chunks=4, dim=1)

        input_gate = torch.sigmoid(i_conv + self.W_ci * C_prev)
        forget_gate = torch.sigmoid(f_conv + self.W_cf * C_prev)

        # Current Cell output
        C = forget_gate * C_prev + input_gate * self.activation(C_conv)

        output_gate = torch.sigmoid(o_conv + self.W_co * C)

        # Current Hidden State
        H = output_gate * self.activation(C)

        return H, C

class ConvLSTM(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, padding, activation, frame_size, return_sequence=False):
        super(ConvLSTM, self).__init__()

        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        # self.device = torch.device('cpu')
        self.out_channels = out_channels
        self.return_sequence = return_sequence

        # We will unroll this over time steps
        self.convLSTMcell = ConvLSTMCell(in_channels, out_channels, kernel_size, padding, activation, frame_size)

    def forward(self, X):
        # X is a frame sequence (batch_size, seq_len, num_channels, height, width)

        # Get the dimensions
        batch_size, seq_len, channels, height, width = X.size()

        # Initialize output
        output = torch.zeros(batch_size, seq_len, self.out_channels, height, width, device=self.device)

        # Initialize Hidden State
        H = torch.zeros(batch_size, self.out_channels, height, width, device=self.device)

        # Initialize Cell Input
        C = torch.zeros(batch_size, self.out_channels, height, width, device=self.device)

        # Unroll over time steps
        for time_step in range(seq_len):
            H, C = self.convLSTMcell(X[:, time_step, ...], H, C)
            # H, C = self.convLSTMcell(X, H, C)
            output[:, time_step, ...] = H

        if not self.return_sequence:
            output = torch.squeeze(output[:, -1, ...], dim=1)

        return output

class ConvBLSTM(nn.Module):
    def __init__(self, in_channels, out_channels,
                 kernel_size, padding, activation, frame_size, return_sequence=False):
        super(ConvBLSTM, self).__init__()
        self.return_sequence = return_sequence
        self.forward_cell = ConvLSTM(in_channels, out_channels//2, 
                                     kernel_size, padding, activation, frame_size, return_sequence=True)
        self.backward_cell = ConvLSTM(in_channels, out_channels//2, 
                                     kernel_size, padding, activation, frame_size, return_sequence=True)

    def forward(self, x):
        y_out_forward = self.forward_cell(x)
        reversed_idx = list(reversed(range(x.shape[1])))
        y_out_reverse = self.backward_cell(x[:, reversed_idx, ...])[:, reversed_idx, ...]
        output = torch.cat((y_out_forward, y_out_reverse), dim=2)
        if not self.return_sequence:
            output = torch.squeeze(output[:, -1, ...], dim=1)
        return output

class BCDUNet(nn.Module):
    def __init__(self, input_dim=3, output_dim=3, num_filter=64, frame_size=(256, 256), bidirectional=False, norm='instance'):
        super(BCDUNet, self).__init__()
        self.num_filter = num_filter
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout = nn.Dropout(0.5)
        self.frame_size = np.array(frame_size)

        if norm == 'instance':
            norm_layer = nn.InstanceNorm2d
        else:
            norm_layer = nn.BatchNorm2d

        def conv_block(in_channels, out_channels):
            return nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1),
                norm_layer(out_channels),
                nn.ReLU(inplace=True),
                nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1),
                norm_layer(out_channels),
                nn.ReLU(inplace=True)
            )
        
        # Encoder
        resnet34 = models.resnet34(weights=ResNet34_Weights.DEFAULT)
        filters = [64, 128, 256, 512]
        
        self.res_input = resnet34.conv1
        self.res_bn1 = nn.BatchNorm2d(64)
        self.res_bn2 = nn.BatchNorm2d(128)
        self.res_bn3 = nn.BatchNorm2d(256)
        self.res_bn4 = nn.BatchNorm2d(512)
        self.res_relu = nn.ReLU(inplace=False)
        self.res_maxpool = resnet34.maxpool
        self.encoder1 = resnet34.layer2
        self.encoder2 = resnet34.layer3
        self.encoder3 = resnet34.layer4

        self.bridge = nn.Sequential(
            nn.Conv2d(filters[3], filters[3]*2, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(filters[3]*2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
            
        )

        self.upconv3 = nn.ConvTranspose2d(num_filter * 8, num_filter * 4, kernel_size=2, stride=2)
        self.upconv2 = nn.ConvTranspose2d(num_filter * 4, num_filter * 2, kernel_size=2, stride=2)
        self.upconv1 = nn.ConvTranspose2d(num_filter * 2, num_filter, kernel_size=2, stride=2)
        self.upconv0 = nn.ConvTranspose2d(num_filter, output_dim, kernel_size=2, stride=2)

        self.conv3m = conv_block(num_filter * 8, num_filter * 4)
        self.conv2m = conv_block(num_filter * 4, num_filter * 2)
        self.conv1m = conv_block(num_filter * 2, num_filter)

        self.conv0 = nn.Conv2d(output_dim, output_dim, kernel_size=1)

        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

        if bidirectional:
            self.clstm1 = ConvBLSTM(num_filter*4, num_filter*2, (3, 3), (1,1), 'tanh', list(self.frame_size//4))
            self.clstm2 = ConvBLSTM(num_filter*2, num_filter, (3, 3), (1,1), 'tanh', list(self.frame_size//2))
            self.clstm3 = ConvBLSTM(num_filter, num_filter//2, (3, 3), (1,1), 'tanh', list(self.frame_size))
        else:
            self.clstm1 = ConvLSTM(num_filter*4, num_filter*2, (3, 3), (1,1), 'tanh', list(self.frame_size//4))
            self.clstm2 = ConvLSTM(num_filter*2, num_filter, (3, 3), (1,1), 'tanh', list(self.frame_size//2))
            self.clstm3 = ConvLSTM(num_filter, num_filter//2, (3, 3), (1,1), 'tanh', list(self.frame_size))

    def forward(self, x):
        N = self.frame_size

        ## Encoder 
        conv1 = self.res_input(x)
        conv1 = self.res_relu(conv1)
        conv1 = self.res_bn1(conv1)
        conv2 = self.encoder1(conv1)
        conv2 = self.res_relu(conv2)
        conv2 = self.res_bn2(conv2)
        conv3 = self.encoder2(conv2)
        conv3 = self.res_bn3(conv3)
        conv4 = self.encoder3(conv3)
        conv4 = self.res_bn4(conv4)

        ## Decoder
        upconv3 = self.upconv3(conv4)
        upconv32 = upconv3.unsqueeze(0).transpose(0, 1)
        upconv32 = torch.cat([upconv32] * 2, dim=1)
        concat3 = self.clstm1(upconv32)
        concat3 = torch.cat((concat3, concat3), 1)
        concat3 = torch.cat((conv3, concat3), 1)
        conv3m = self.conv3m(concat3)
        conv3m = self.relu(conv3m)

        upconv2 = self.upconv2(conv3m)
        upconv22 = upconv2.unsqueeze(0).transpose(0, 1)
        upconv22 = torch.cat([upconv22] * 2, dim=1)
        concat2 = self.clstm2(upconv22)
        concat2 = torch.cat((concat2, concat2), 1)
        concat2 = torch.cat((conv2, concat2), 1)
        conv2m = self.conv2m(concat2)
        conv2m = self.relu(conv2m)

        upconv1 = self.upconv1(conv2m)
        upconv12 = upconv1.unsqueeze(0).transpose(0, 1)
        upconv22 = torch.cat([upconv22] * 2, dim=1)
        concat1 = self.clstm3(upconv12)
        concat1 = torch.cat((concat1, concat1), 1)
        concat1 = torch.cat((conv1, concat1), 1)
        conv1m = self.conv1m(concat1)
        conv1m = self.relu(conv1m)

        upconv0 = self.upconv0(conv1m)
        conv0 = self.conv0(upconv0)

        return conv0

class Lung_Dataset(Dataset):
    '''
    x: Features.
    y: Targets, if none, do prediction.
    '''
    def __init__(self, x, y=None):
        if y is None:
            self.y = y
        else:
            self.y = torch.FloatTensor(y)
        self.x = torch.FloatTensor(x)

    def __getitem__(self, idx):
        if self.y is None:
            return self.x[idx]
        else:
            return self.x[idx], self.y[idx]

    def __len__(self):
        return len(self.x)
    
def predict(test_loader, model, device):
        model.eval() # Set your model to evaluation mode.
        preds = []
        # pbar = tqdm(range(len(test_loader)))
        # pbar.set_description("Predicting")
        for x in test_loader:
            x = x.to(device)                        
            with torch.no_grad():                   
                pred = model(x)                     
                preds.append(pred.detach().cpu())  
                # pbar.update(1) 
        preds = torch.cat(preds, dim=0).numpy()  
        return preds
    
def edge_clean(matrix):
    # Clear the top and bottom edges
    matrix[:5, :] = 0
    matrix[-5:, :] = 0
    
    # Clear the left and right edges
    matrix[:, :5] = 0
    matrix[:, -5:] = 0

def hole_filler(Estimated_lung, Estimated_lung2):
    Lung_mask = np.zeros((Estimated_lung.shape[0], 512, 512))
    Filled_lung = np.zeros((Estimated_lung.shape[0], 512, 512))
    # for k in tqdm(range(Estimated_lung.shape[0]), desc="Filling"):
    for k in range(Estimated_lung.shape[0]):
        edge_clean(Estimated_lung[k])
        edge_clean(Estimated_lung2[k])
        Estimated_lung[k] = binary_erosion(Estimated_lung[k], iterations=5)
        Estimated_lung2[k] = binary_erosion(Estimated_lung2[k], iterations=5)
        noFill = np.zeros((512, 512))
        visited = np.zeros((512, 512))
        queue = deque([(0, 0)])
        while queue:
            node = queue.popleft()
            if visited[node[0]][node[1]] == 0:
                visited[node[0]][node[1]] = 1
                noFill[node[0]][node[1]] = 1
                for d in [(-1, 0), (0, 1), (0, -1), (1, 0)]:
                    if node[0]+d[0] >= 0 and node[0]+d[0] < 512 and node[1]+d[1] >= 0 and node[1]+d[1] < 512:
                        if Estimated_lung[k][node[0]+d[0]][node[1]+d[1]] == 0:
                            queue.append((node[0]+d[0], node[1]+d[1]))
        Estimated_lung[k][noFill != 1] = 1
        Lung_mask[k] = np.subtract(Estimated_lung[k], Estimated_lung2[k])
        Lung_mask[k] = binary_dilation(Lung_mask[k], iterations=5)
        Lung_mask[k] = binary_erosion(Lung_mask[k], iterations=5)
        Filled_lung[k] = copy.deepcopy(Lung_mask[k])
        noFill = np.zeros((512, 512))
        visited = np.zeros((512, 512))
        # print(last_node)
        queue = deque([(0, 0)])
        while queue:
            node = queue.popleft()
            if visited[node[0]][node[1]] == 0:
                visited[node[0]][node[1]] = 1
                noFill[node[0]][node[1]] = 1
                for d in [(-1, 0), (0, 1), (0, -1), (1, 0)]:
                    if node[0]+d[0] >= 0 and node[0]+d[0] < 512 and node[1]+d[1] >= 0 and node[1]+d[1] < 512:
                        if Filled_lung[k][node[0]+d[0]][node[1]+d[1]] == 0:
                            queue.append((node[0]+d[0], node[1]+d[1]))
        Filled_lung[k][noFill != 1] = 1
    # Lung_mask = np.subtract(Estimated_lung, Estimated_lung2)
    return Estimated_lung, Estimated_lung2, Lung_mask, Filled_lung

def hole_filler_improved(Estimated_lung, Estimated_lung2):
    Lung_mask = np.zeros((Estimated_lung.shape[0], 512, 512))
    Filled_lung = np.zeros((Estimated_lung.shape[0], 512, 512))
    # for k in tqdm(range(Estimated_lung.shape[0]), desc="Fill"):
    for k in range(Estimated_lung.shape[0]):
        edge_clean(Estimated_lung[k])
        edge_clean(Estimated_lung2[k])
        Estimated_lung[k] = binary_erosion(Estimated_lung[k], iterations=5)
        Estimated_lung2[k] = binary_erosion(Estimated_lung2[k], iterations=5)
        noFill = np.zeros((512, 512))
        visited = np.zeros((512, 512))
        queue = deque([(0, 0)])
        last_node = [(0, 0)]
        while queue:
            node = queue.popleft()
            if visited[node[0]][node[1]] == 0:
                last_node = [node]
                visited[node[0]][node[1]] = 1
                noFill[node[0]][node[1]] = 1
                for d in [(-1, 0), (0, 1), (0, -1), (1, 0)]:
                    if node[0]+d[0] >= 0 and node[0]+d[0] < 512 and node[1]+d[1] >= 0 and node[1]+d[1] < 512:
                        if Estimated_lung[k][node[0]+d[0]][node[1]+d[1]] == 0:
                            queue.append((node[0]+d[0], node[1]+d[1]))
        Estimated_lung[k][noFill != 1] = 1
        Lung_mask[k] = np.subtract(Estimated_lung[k], Estimated_lung2[k])
        Lung_mask[k] = binary_dilation(Lung_mask[k], iterations=5)
        Lung_mask[k] = binary_erosion(Lung_mask[k], iterations=5)
        Filled_lung[k] = copy.deepcopy(Lung_mask[k])
        noFill[Estimated_lung[k] == Estimated_lung2[k]] = 1
        inside_holes = np.zeros((512, 512))
        inside_holes[(noFill != 1)] = 1
        # print(inside_holes.shape)
        inside_holes = binary_dilation(inside_holes, iterations=5)
        inside_holes = binary_erosion(inside_holes, iterations=5)
        inside_holes = binary_fill_holes(inside_holes)
        # fig, ax = plt.subplots(1, 1, figsize=[5, 5])
        # ax.imshow(inside_holes, cmap='gray')
        # plt.show()
        noFill[inside_holes == 1] = 0
        visited[Estimated_lung[k] == Estimated_lung2[k]] = 1
        # queue = deque(last_node)
        # while queue:
        #     node = queue.popleft()
        #     if visited[node[0]][node[1]] == 2:
        #         visited[node[0]][node[1]] = 1
        #         noFill[node[0]][node[1]] = 1
        #         for d in [(-1, 0), (0, 1), (0, -1), (1, 0)]:
        #             if node[0]+d[0] >= 0 and node[0]+d[0] < 512 and node[1]+d[1] >= 0 and node[1]+d[1] < 512:
        #                 if Filled_lung[k][node[0]+d[0]][node[1]+d[1]] == 0:
        #                     queue.append((node[0]+d[0], node[1]+d[1]))
        Filled_lung[k][noFill != 1] = 1
    # Lung_mask = np.subtract(Estimated_lung, Estimated_lung2)
    return Estimated_lung, Estimated_lung2, Lung_mask, Filled_lung

def hole_filler_convex_hull(Estimated_lung, Estimated_lung2):
    Lung_mask = np.zeros((Estimated_lung.shape[0], 512, 512))
    Filled_lung = np.zeros((Estimated_lung.shape[0], 512, 512))
    # for k in tqdm(range(Estimated_lung.shape[0]), desc="Fill"):
    for k in range(Estimated_lung.shape[0]):
        edge_clean(Estimated_lung[k])
        edge_clean(Estimated_lung2[k])
        Estimated_lung[k] = binary_erosion(Estimated_lung[k], iterations=5)
        Estimated_lung2[k] = binary_erosion(Estimated_lung2[k], iterations=5)
        noFill = np.zeros((512, 512))
        visited = np.zeros((512, 512))
        queue = deque([(0, 0)])
        while queue:
            node = queue.popleft()
            if visited[node[0]][node[1]] == 0:
                visited[node[0]][node[1]] = 1
                noFill[node[0]][node[1]] = 1
                for d in [(-1, 0), (0, 1), (0, -1), (1, 0)]:
                    if node[0]+d[0] >= 0 and node[0]+d[0] < 512 and node[1]+d[1] >= 0 and node[1]+d[1] < 512:
                        if Estimated_lung[k][node[0]+d[0]][node[1]+d[1]] == 0:
                            queue.append((node[0]+d[0], node[1]+d[1]))
        Estimated_lung[k][noFill != 1] = 1
        Lung_mask[k] = np.subtract(Estimated_lung[k], Estimated_lung2[k])
        Lung_mask[k] = binary_dilation(Lung_mask[k], iterations=5)
        Lung_mask[k] = binary_erosion(Lung_mask[k], iterations=5)
        Filled_lung[k] = copy.deepcopy(Lung_mask[k])
        noFill[Estimated_lung[k] == Estimated_lung2[k]] = 1
        inside_holes = np.zeros((512, 512))
        inside_holes[(noFill != 1)] = 1
        inside_holes = binary_dilation(inside_holes, iterations=5)
        inside_holes = binary_erosion(inside_holes, iterations=5)
        inside_holes = binary_fill_holes(inside_holes)
        noFill[inside_holes == 1] = 0
        # try:
        #     Filled_lung[k], new_start = convex_hull_helper(Filled_lung[k])
        #     # print(new_start)
        #     queue = deque(new_start)
        #     while queue:
        #         node = queue.popleft()
        #         # print(node)
        #         if visited[node[0]][node[1]] == 0:
        #             visited[node[0]][node[1]] = 1
        #             noFill[node[0]][node[1]] = 0
        #             for d in [(-1, 0), (0, 1), (0, -1), (1, 0)]:
        #                 if node[0]+d[0] >= 0 and node[0]+d[0] < 512 and node[1]+d[1] >= 0 and node[1]+d[1] < 512:
        #                     if Filled_lung[k][node[0]+d[0]][node[1]+d[1]] == 0:
        #                         queue.append((node[0]+d[0], node[1]+d[1]))
        #     Filled_lung[k][noFill != 1] = 1
        #     Filled_lung[k] = binary_fill_holes(Filled_lung[k])
        # except:
        #     pass
    return Estimated_lung, Estimated_lung2, Lung_mask, Filled_lung

def init_segment_model():
    input_dim = 3
    output_dim = 3
    num_filter = 64
    frame_size = (256, 256)
    bidirectional = True
    norm = 'instance'
    batch_size = 2
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("cuda is available: ", torch.cuda.is_available())
    
    model = BCDUNet(input_dim, output_dim, num_filter, frame_size, bidirectional, norm).to(device)
    model.load_state_dict(torch.load('E:/LUNA/Lung_segmentation/model.pth', map_location=torch.device(device)))
    return model, device
    


def segment(ct_scan, model, device):
    result = []
    ct_scan[ct_scan < -1000] = -1000
    ct_scan[ct_scan> 1000] = 1000
    # augmentation, transpose and duplicate 3 times
    ct_expand  = np.expand_dims(ct_scan, axis=3)
    ct_scaled = ct_expand / 255
    ct_scaled_tnsps = torch.tensor(ct_scaled).transpose(1, 3)
    # print(ct_scaled_tnsps.shape)
    ct_augmented = torch.cat([ct_scaled_tnsps] * 3, dim=1).numpy()
    
    input_dim = 3
    output_dim = 3
    num_filter = 64
    frame_size = (256, 256)
    bidirectional = True
    norm = 'instance'
    batch_size = 2
    
    test_dataset = Lung_Dataset(ct_augmented)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    predictions = predict(test_loader, model, device) 
    
    # Post-processing
    predictions = np.squeeze(predictions)
    if(len(predictions.shape) == 3):
        predictions = np.expand_dims(predictions, axis=0)
    # print(predictions.shape)
    predictions = torch.tensor(predictions).transpose(1, 3)
    predictions = np.where(predictions<0.5, 1, 0)
    Estimated_lung = predictions[:,:,:,0]
    Estimated_lung2 = copy.deepcopy(Estimated_lung)
    # plt.imshow(Estimated_lung[0], cmap='gray')
    # plt.show()
    
    # Estimated_lung, Estimated_lung2, Lung_mask, Filled_lung = hole_filler(Estimated_lung, Estimated_lung2)
    # Estimated_lung, Estimated_lung2, Lung_mask, Filled_lung = hole_filler_improved(Estimated_lung, Estimated_lung2)
    Estimated_lung, Estimated_lung2, Lung_mask, Filled_lung = hole_filler_convex_hull(Estimated_lung, Estimated_lung2)
    Filled_lung = binary_dilation(Filled_lung, iterations=20)
    # Estimated_lung, Estimated_lung2, Lung_mask, Filled_lung = hole_filler_dilation(Estimated_lung, Estimated_lung2)
    
    slicesize = len(ct_expand)
    for idx in range(slicesize):
        seg_result =  np.squeeze(ct_expand[idx])*Filled_lung[idx]
        seg_result = seg_result.astype(int)
        seg_result[seg_result == 0] = 1000
        result.append(seg_result)
        # fig, ax = plt.subplots(1, 1, figsize=[5, 5])
        # ax.imshow(seg_result, cmap='gray')
        # plt.show()
    # plt.imshow(Estimated_lung[0], cmap='gray')
    # plt.show()
    # plt.imshow(Estimated_lung2[0], cmap='gray')
    # plt.show()
    # plt.imshow(Lung_mask[0], cmap='gray')
    # plt.show()
    # plt.imshow(Filled_lung[0], cmap='gray')
    # plt.show()
    result = np.stack(result, axis=0)
    return result


def convex_hull_helper(ct_scan):
    # converted = np.where(te_data!=1000, 1, 0)
    coords = np.where(ct_scan==1)
    coords = np.array(list(zip(coords[0], coords[1])))
    # print(coords)
    hull = ConvexHull(coords)
    points_within = set()
    hull_len = len(coords[hull.vertices])
    for j in range(hull_len):
        for m in list(bresenham(coords[hull.vertices][j][0], coords[hull.vertices][j][1], coords[hull.vertices][(j+1)%hull_len][0], coords[hull.vertices][(j+1)%hull_len][1])):
            points_within.add(m)
    points_within = list(points_within)
    # print(list(points_within))
    # plt.imshow(converted, cmap='gray')
    # plt.show()
    # plt.imshow(converted, cmap='gray')
    # for j in tqdm(points_within, desc='Marking progress'):
    #     plt.scatter(j[1], j[0], c='r', marker='o', s=1)
    # plt.show()
    # for j in points_within:
    #     ct_scan[j[0]][j[1]] = 1
    for j in points_within:
        for k in range(-15, 15):
            try:
                ct_scan[j[0]-k][j[1]] = 1
                ct_scan[j[0]+k][j[1]] = 1
                ct_scan[j[0]][j[1]-k] = 1
                ct_scan[j[0]][j[1]+k] = 1
            except:
                pass
    point_avg = np.mean(points_within, axis=0)
    point_avg = [tuple(np.round(point_avg).astype(int))]
    # for j in tqdm(points_within, desc='Marking progress'):
    #     plt.scatter(j[1], j[0], c='r', marker='o', s=1)
    # plt.imshow(converted, cmap='gray')
    # plt.show()
    return ct_scan, point_avg

# import SimpleITK as sitk
# import matplotlib.pyplot as plt
# data = sitk.ReadImage("C:/Users/oplab/Desktop/Luna16_data/Luna16_img/subset0/1.3.6.1.4.1.14519.5.2.1.6279.6001.105756658031515062000744821260.mhd")
# ct_scan = sitk.GetArrayFromImage(data)[60:61]
# # ct_scan = sitk.GetArrayFromImage(data)[13:17]
# # ct_scan = sitk.GetArrayFromImage(data)[13:17]
# # result = segment(np.expand_dims(ct_scan[0:2], axis=0))
if isLungSegmentProcess:
    model, device = init_segment_model()
# result = segment(ct_scan, model, device)
# for i in range(len(result)):
#     plt.imshow(ct_scan[i], cmap='gray')
#     plt.show()
#     plt.imshow(result[i], cmap='gray')
#     plt.show()

In [19]:
idx = 0
uid_ndcnt = 0

record = {'Train': {'Benign': 0, 'Malignancy': 0}, 'Valid': {'Benign':  0, 'Malignancy': 0}, 'Test': {'Benign':  0, 'Malignancy':  0}, 'Total': 0}

malig_rotation, benign_rotation = 0, 0

for uid in candidate_dict:

    mhd_path = glob.glob('E:/LUNA/Luna16_AugData/subset*/{}.mhd'.format(uid))

    ct_mhd = sitk.ReadImage(mhd_path)

    if ct_mhd.GetDimension()==4 and ct_mhd.GetSize()[3]==1:
        ct_mhd = ct_mhd[...,0]

    hu_a = np.array(sitk.GetArrayFromImage(ct_mhd), dtype=np.float32)
    
    if isLungSegmentProcess:
        aug_result = segment(hu_a, model, device)

    origin_xyz = XyzTuple(*ct_mhd.GetOrigin())

    vxSize_xyz = XyzTuple(*ct_mhd.GetSpacing())

    direction_a = np.array(ct_mhd.GetDirection()).reshape(3, 3)

    mask_a, bbox_result = buildAnnotationMask(candidate_dict[uid], hu_a, origin_xyz, vxSize_xyz, direction_a)
    
    #print("bbox_result:", bbox_result)

    lower_bound = 0

    upper_bound = len(hu_a)

    current_slice = 0

    uid_ndcnt = 0

    for candidateInfo_tup in candidate_dict[uid]:

        center_irc = xyz2irc(candidateInfo_tup.center_xyz, origin_xyz, vxSize_xyz, direction_a)

        index, row, column = center_irc
        
        
        hu_list = []
        mask_list = []
        
        for i in range(candidateInfo_tup.cbbox[0], candidateInfo_tup.cbbox[1]):
            
            
            con_mask = candidateInfo_tup[6][i - candidateInfo_tup.cbbox[0]]
            #print("row: ", row, " column: ", column, " bbox: ", bbox_result[uid_ndcnt])
            #print(con_mask)
            if con_mask == 'nan':
                #print('found nan')
                continue
            #new_con_mask = [bbox_result[uid_ndcnt][0], bbox_result[uid_ndcnt][1], con_mask[2], con_mask[3]]
            new_con_mask = [con_mask[0], con_mask[1], con_mask[2], con_mask[3]]
            current_slice = i
            # current_slice = index + i
            # if index + i < lower_bound:
            #     current_slice = lower_bound
            # if index + i > upper_bound:
            #     current_slice = upper_bound
            
            gen_path = ""
            is_training = False
            doTest = True
            
            if candidateInfo_tup.isMal_bool:
                if generate['Train']['Malignancy'] > 0 and malig_rotation % 10 < 8:
                    gen_path = "E:/LUNA/Autolabel/1126/yolo_lung_aug/Train"
                    generate['Train']['Malignancy'] -= 1
                    is_training = True
                    record['Train']['Malignancy'] += 6
                    record['Total'] += 5
                else:
                    if generate['Valid']['Malignancy'] > 0 and malig_rotation % 10 == 8:
                        gen_path = "E:/LUNA/Autolabel/1126/yolo_lung_aug/Valid"
                        generate['Valid']['Malignancy'] -= 1
                        record['Valid']['Malignancy'] += 1
                        
                    else:
                        gen_path = "E:/LUNA/Autolabel/1126/yolo_lung_aug/Test"
                        generate['Test']['Malignancy'] -= 1
                        record['Test']['Malignancy'] += 1
                        doTest = True
                malig_rotation += 1                
            else:
                if generate['Train']['Benign'] > 0 and benign_rotation % 10 < 8:
                    gen_path = "E:/LUNA/Autolabel/1126/yolo_lung_aug/Train"
                    generate['Train']['Benign'] -= 1
                    is_training = True
                    record['Train']['Benign'] += 6
                    record['Total'] += 5
                else:
                    if generate['Valid']['Benign'] > 0 and benign_rotation % 10 == 8:
                        gen_path = "E:/LUNA/Autolabel/1126/yolo_lung_aug/Valid"
                        generate['Valid']['Benign'] -= 1
                        record['Valid']['Benign'] += 1
                    else:
                        gen_path = "E:/LUNA/Autolabel/1126/yolo_lung_aug/Test"
                        generate['Test']['Benign'] -= 1
                        record['Test']['Benign'] += 1
                        doTest = True
                benign_rotation += 1
            record['Total'] += 1
            
            if doTest:
            
                # np2Png(aug_result[current_slice], (gen_path + "/images/{}-({})-[{}]_{}_{}.png").format(idx, i, 0, uid_ndcnt, uid))
                #np2Png(mask_a[current_slice].astype(float), "C:/LUNA/U-Det/Data/masks-photo/{}-({})_{}_{}.png".format(idx, i, uid_ndcnt, uid))
                #print("Ans:")
                #np2MarkPng(hu_a[current_slice], "E:/LUNA/Autolabel/comparison/{}-({})_{}_{}_true.png".format(idx, i, uid_ndcnt, uid), row, column)
                #print("result:")
                #bboxOnImg((gen_path + "/images/{}-({})-[{}]_{}_{}.png").format(idx, i, 0, uid_ndcnt, uid), new_con_mask)            
                with open((gen_path + "/labels/{}-({})-[{}]_{}_{}.txt").format(idx, i, 0, uid_ndcnt, uid), 'w') as txt:
                    txt.write(str(int(candidateInfo_tup[2])) + ' ')
                    for bbox_value in new_con_mask:
                        txt.write(str(bbox_value) + ' ')
                
                if is_training:
                    for j in range(1, 4):
                        #np2Png(np.rot90(aug_result[current_slice], k = j), (gen_path + "/images/{}-({})-[{}]_{}_{}.png").format(idx, i, j, uid_ndcnt, uid))
                        
                        degrees = 90 * j
                        [x_rotate, y_rotate, width_rotate, height_rotate] = new_con_mask
                        
                        if degrees == 90:
                            rotation_matrix = np.array([[0, 1], [-1, 0]])
                        elif degrees == 180:
                            rotation_matrix = np.array([[-1, 0], [0, -1]])
                        elif degrees == 270:
                            rotation_matrix = np.array([[0, -1], [1, 0]])
                        else:
                            raise ValueError("Invalid rotation degrees. Supported values are 90, 180, and 270.")

                        # Rotate the rectangle's center (x, y)
                        rotated_center = np.dot(rotation_matrix, np.array([x_rotate, y_rotate]))
                        #print("rotate center: ", rotated_center)

                        # Update the rectangle's position
                        x_rotated, y_rotated = rotated_center
                        if j == 1:
                            rotate_new_con_mask = [x_rotated, 1 + y_rotated, height_rotate, width_rotate]
                        elif j == 2:
                            rotate_new_con_mask = [1 + x_rotated, 1 + y_rotated, width_rotate, height_rotate]
                        elif j == 3:
                            rotate_new_con_mask = [1 + x_rotated, y_rotated, height_rotate, width_rotate]
                            
                        with open((gen_path + "/labels/{}-({})-[{}]_{}_{}.txt").format(idx, i, j, uid_ndcnt, uid), 'w') as txt:
                            txt.write(str(int(candidateInfo_tup[2])) + ' ')
                            for bbox_value in rotate_new_con_mask:
                                txt.write(str(bbox_value) + ' ')
                        
                        #bboxOnImg((gen_path + "/images/{}-({})-[{}]_{}_{}.png").format(idx, i, j, uid_ndcnt, uid), rotate_new_con_mask)            

                    
                    #np2Png(np.flip(aug_result[current_slice], axis=1), (gen_path + "/images/{}-({})-[{}]_{}_{}.png").format(idx, i, 4, uid_ndcnt, uid)) # horizontal
                    rotate_new_con_mask = [1 - new_con_mask[0], new_con_mask[1], new_con_mask[2], new_con_mask[3]]
                    with open((gen_path + "/labels/{}-({})-[{}]_{}_{}.txt").format(idx, i, 4, uid_ndcnt, uid), 'w') as txt:
                        txt.write(str(int(candidateInfo_tup[2])) + ' ')
                        for bbox_value in rotate_new_con_mask:
                            txt.write(str(bbox_value) + ' ')
                                
                    #bboxOnImg((gen_path + "/images/{}-({})-[{}]_{}_{}.png").format(idx, i, 4, uid_ndcnt, uid), rotate_new_con_mask)
                    
                    rotate_new_con_mask = [new_con_mask[0], 1 - new_con_mask[1], new_con_mask[2], new_con_mask[3]]
                    #np2Png(np.flip(aug_result[current_slice], axis=0), (gen_path + "/images/{}-({})-[{}]_{}_{}.png").format(idx, i, 5, uid_ndcnt, uid)) # vertical
                    with open((gen_path + "/labels/{}-({})-[{}]_{}_{}.txt").format(idx, i, 5, uid_ndcnt, uid), 'w') as txt:
                        txt.write(str(int(candidateInfo_tup[2])) + ' ')
                        for bbox_value in rotate_new_con_mask:
                            txt.write(str(bbox_value) + ' ')
                    
                    #bboxOnImg((gen_path + "/images/{}-({})-[{}]_{}_{}.png").format(idx, i, 5, uid_ndcnt, uid), rotate_new_con_mask)

            print(record)
            
            #hu_list.append(hu_a[current_slice])
            #mask_list.append(mask_a[current_slice])
        #tr_data = np.stack(hu_list)
        #gt_data = np.stack(mask_list)
        #np.save('C:/LUNA/U-Det/Data/imgs/images_{}_{}_{}.npy'.format(idx, uid_ndcnt, uid), tr_data)
        #np.save('C:/LUNA/U-Det/Data/masks/masks_{}_{}_{}.npy'.format(idx, uid_ndcnt, uid), gt_data)
        idx += 1
        uid_ndcnt += 1


{'Train': {'Benign': 0, 'Malignancy': 6}, 'Valid': {'Benign': 0, 'Malignancy': 0}, 'Test': {'Benign': 0, 'Malignancy': 0}, 'Total': 6}
{'Train': {'Benign': 0, 'Malignancy': 12}, 'Valid': {'Benign': 0, 'Malignancy': 0}, 'Test': {'Benign': 0, 'Malignancy': 0}, 'Total': 12}
{'Train': {'Benign': 0, 'Malignancy': 18}, 'Valid': {'Benign': 0, 'Malignancy': 0}, 'Test': {'Benign': 0, 'Malignancy': 0}, 'Total': 18}
{'Train': {'Benign': 0, 'Malignancy': 24}, 'Valid': {'Benign': 0, 'Malignancy': 0}, 'Test': {'Benign': 0, 'Malignancy': 0}, 'Total': 24}
{'Train': {'Benign': 0, 'Malignancy': 30}, 'Valid': {'Benign': 0, 'Malignancy': 0}, 'Test': {'Benign': 0, 'Malignancy': 0}, 'Total': 30}
{'Train': {'Benign': 0, 'Malignancy': 36}, 'Valid': {'Benign': 0, 'Malignancy': 0}, 'Test': {'Benign': 0, 'Malignancy': 0}, 'Total': 36}
{'Train': {'Benign': 0, 'Malignancy': 42}, 'Valid': {'Benign': 0, 'Malignancy': 0}, 'Test': {'Benign': 0, 'Malignancy': 0}, 'Total': 42}
{'Train': {'Benign': 0, 'Malignancy': 48}, 

In [20]:
print(generate)

{'Train': {'Benign': 0, 'Malignancy': 0}, 'Valid': {'Benign': 0, 'Malignancy': 0}, 'Test': {'Benign': 0, 'Malignancy': 0}}
